In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main-2'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Mounted at /content/gdrive/


In [2]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [3]:
!pwd

/content/gdrive/My Drive/IDLSProject-main


In [4]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split

data_dir = './data/pytorch'

with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)
    
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)


test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

#drop rows
test.drop(test.tail(227).index,inplace = True)
val.drop(val.tail(228).index,inplace = True)
test.shape, val.shape

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape, val_y.shape)


(83000, 502) (21975, 502)
torch.Size([10760, 1]) torch.Size([10760, 1])


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 10000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]
criterion = torch.nn.BCELoss().to(device)

In [6]:
data_dir = './lstm-cnn-amazon-hyperband-trails/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./lstm-cnn-amazon-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [7]:
import torch
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,trial,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_size = trial.suggest_int("hidden_size",32,256)
        embed_size = trial.suggest_int("embed_size",16,128)
        filter_size = trial.suggest_int("filter_size",50,200)
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(input_size = embed_size, hidden_size = hidden_size,num_layers=1,batch_first=True)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],hidden_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],hidden_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],hidden_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        self.dense = nn.Linear(3*filter_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x,target):
    
        embeds = self.embedding(x)
        # print('embedds')
        # print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        # print('lstm output')
        # print(lstm_out.shape)
        lstm_out = lstm_out.clone()
        lstm_out.unsqueeze_(1)
        # print('lstm output after')
        # print(lstm_out.shape)
        x1 = torch.tanh(self.dropout(self.conv1(lstm_out))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(lstm_out))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(lstm_out))).squeeze(3)
        # print('x1.shape')
        # print(x1.shape)
        f1 = self.mp1(x1).squeeze(2)
        # print('f1')
        # print(f1.shape)
        f2 = self.mp2(x2).squeeze(2)
        # print('f2')
        # print(f2.shape)
        f3 = self.mp3(x3).squeeze(2)
        # print('f3')
        # print(f3.shape)
        hidden = torch.cat([f1,f2,f3],dim=1)
        logits = self.dense(hidden)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy
        # return self.sig(out)

In [8]:
import time
import numpy as np
import torch.optim as optim

def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = Combine(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
  trial.set_user_attr(key="best_model", value=model)
  optimizer = optim.Adam(model.parameters())
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  counter = 0
  CNN_acc = []
  CNN_valacc = []
  train_loss_epoch = []
  train_acc_epoch = []
  val_loss_epoch = []
  val_acc_epoch = []
  time_epoch = []
  model.train()
  final_val_acc = 0
  for e in range(epochs):
    start_time = time.time()
    train_loss = []
    train_acc = []
    for inputs, labels in train_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        model.zero_grad()
        logits, accuracy = model(inputs,labels)
        loss = criterion(logits,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        train_loss.append(loss.item())
        train_acc.append(accuracy.item()*100/batch_size)
        if counter%100==0:
            print("Epoch: {}/{}".format(e,epochs),
                         "\tIteration: {}".format(counter),
                         "\t\tTrain Loss: {:.3f}".format(loss.item()),
                         "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
            CNN_acc.append(accuracy.item()*100/batch_size)
        counter += 1
    train_loss_epoch.append(np.round(np.mean(train_loss), 3))
    train_acc_epoch.append(np.round(np.mean(train_acc), 3))
    print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))

    with torch.no_grad():
        model.eval()
        val_acc = []
        val_loss = []
        for inputs, labels in val_dl:
            inputs_val, labels_val = inputs.cuda(), labels.cuda()
            logits_val,accuracy_val = model(inputs_val,labels_val)
            loss_val = criterion(logits_val,labels_val.float())
            val_acc.append(accuracy_val.item()*100/batch_size)
            val_loss.append(loss_val.item())
        val_loss_epoch.append(np.round(np.mean(val_loss), 3))
        val_acc_epoch.append(np.round(np.mean(val_acc), 3))
        final_val_acc = np.round(np.mean(val_acc), 3)
        print("\t\tVal Loss: {:.3f}".format(np.mean(val_loss)), "\t\tVal Acc: {:.3f}".format(np.mean(val_acc)))
        CNN_valacc.append(np.mean(val_acc))
        model.train()
        
    end_time = time.time()-start_time
    print("Time to train epoch: {0} s".format(end_time))
    time_epoch.append(np.round(end_time, 3))
  write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
  return final_val_acc

In [9]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [10]:
!pip install optuna

     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 11.6 MB/s 
     |████████████████████████████████| 210 kB 72.2 MB/s 
     |████████████████████████████████| 78 kB 7.9 MB/s 
     |████████████████████████████████| 112 kB 74.1 MB/s 
     |████████████████████████████████| 49 kB 7.8 MB/s 
     |████████████████████████████████| 146 kB 67.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=9d095b85212c6fa2a2317e095c3bb5a5ddc9572d1903fb97b0607588a1ec2f0e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [11]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=5, callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-15 04:07:44,712] A new study created in memory with name: no-name-26f95c5f-3eeb-4a02-a0fc-0c85ac9c0d2e


Epoch: 0/10 	Iteration: 0 		Train Loss: 0.682 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.608 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.404 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.397 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.228 	Train Accuracy: 96.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.291 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.172 	Train Accuracy: 96.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.425 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.318 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.125 	Train Accuracy: 96.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.320 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.347 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.214 	Train Accuracy: 92.00
Epoch: 0/10 	Iteration: 1300 		Train Loss: 0.180 	Train Accuracy: 94.00
Epoc

[I 2022-05-15 04:30:14,319] Trial 0 finished with value: 87.787 and parameters: {'optimizer': 'RMSprop', 'lr': 4.429401815530183e-05, 'dropout': 0.2221586224652677, 'hidden_size': 232, 'embed_size': 66, 'filter_size': 143}. Best is trial 0 with value: 87.787.


		Val Loss: 1.811 		Val Acc: 87.787
Time to train epoch: 134.2358009815216 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.662 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.430 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.353 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.393 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.248 	Train Accuracy: 90.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.257 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.193 	Train Accuracy: 96.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.343 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.296 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.087 	Train Accuracy: 98.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.365 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.307 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.182 	Train Accuracy: 94.00

[I 2022-05-15 04:44:21,196] Trial 1 finished with value: 87.713 and parameters: {'optimizer': 'SGD', 'lr': 0.970647313508467, 'dropout': 0.45895112616140743, 'hidden_size': 167, 'embed_size': 118, 'filter_size': 63}. Best is trial 0 with value: 87.787.


...

		Val Loss: 1.478 		Val Acc: 88.139
Time to train epoch: 91.14416933059692 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.733 	Train Accuracy: 20.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.597 	Train Accuracy: 76.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.373 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.340 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.288 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.247 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.180 	Train Accuracy: 94.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.370 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.293 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.164 	Train Accuracy: 94.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.346 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.322 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.179 	Train Accuracy: 92.00

[I 2022-05-15 05:10:00,270] Trial 3 finished with value: 87.889 and parameters: {'optimizer': 'RMSprop', 'lr': 0.007813443444372781, 'dropout': 0.400174578315842, 'hidden_size': 94, 'embed_size': 70, 'filter_size': 105}. Best is trial 2 with value: 88.139.


		Val Loss: 1.469 		Val Acc: 87.889
Time to train epoch: 62.794313192367554 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.735 	Train Accuracy: 20.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.526 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.308 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.377 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.231 	Train Accuracy: 90.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.226 	Train Accuracy: 90.00
Epoch: 0/10 	Iteration: 600 		Train Loss: 0.186 	Train Accuracy: 92.00
Epoch: 0/10 	Iteration: 700 		Train Loss: 0.333 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 800 		Train Loss: 0.296 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 900 		Train Loss: 0.098 	Train Accuracy: 100.00
Epoch: 0/10 	Iteration: 1000 		Train Loss: 0.300 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 1100 		Train Loss: 0.311 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 1200 		Train Loss: 0.226 	Train Accuracy: 88.

[I 2022-05-15 05:22:44,735] Trial 4 finished with value: 87.741 and parameters: {'optimizer': 'SGD', 'lr': 0.4709919111090412, 'dropout': 0.30983404481599963, 'hidden_size': 108, 'embed_size': 125, 'filter_size': 147}. Best is trial 2 with value: 88.139.


		Val Loss: 1.776 		Val Acc: 87.741
Time to train epoch: 76.5160927772522 s
Study statistics: 
  Number of finished trials:  5
  Number of complete trials:  5
Best trial:
  Value:  88.139
  Params: 
    optimizer: Adam
    lr: 0.3541538761638133
    dropout: 0.14804085015911742
    hidden_size: 188
    embed_size: 40
    filter_size: 92


In [12]:
torch.save(best_model.state_dict(),"./lstm-cnn-amazon-hyperband.pth")

In [15]:
criterion = torch.nn.BCELoss().to(device)
def test(model, test_dl, epochs):
    with torch.no_grad():
        model.eval()
        test_acc = []
        test_loss = []
        for inputs, labels in test_dl:
            input_test, labels_test = inputs.cuda(), labels.cuda()
            logits_test,accuracy_test = model(input_test,labels_test)
            loss_test = criterion(logits_test,labels_test.float())
            test_acc.append(accuracy_test.item()*100/batch_size)
            test_loss.append(loss_test.item())
        print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))

In [16]:
test(best_model, test_dl, epochs)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:762: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:926.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Test Loss: 1.594 	Test Acc: 87.491
